In [1]:
# Importa as bibliotecas necessárias
import pandas as pd
from bcb import sgs
from datetime import datetime

# Carrega o dataframe para a memória e converte a coluna 'Data' adequadamente
df_cepea_consulta = pd.read_excel('dags/data/cepea-consulta-20230116155544.xls', header=3,
                                  parse_dates=['Data'], date_parser=lambda x: pd.to_datetime(x, format='%m/%Y'))

df_cepea_consulta

,Data,Valor
0,2021-11-01,"297,66"
1,2021-12-01,"320,90"
2,2022-01-01,"338,46"
3,2022-02-01,"340,29"
4,2022-03-01,"344,71"
5,2022-04-01,"335,06"
6,2022-05-01,"323,10"
7,2022-06-01,"317,96"
8,NaT,"324,41"
9,2022-08-01,"313,39"


In [2]:
# Preenche as datas faltantes com o mês anterior +1, no caso de existir, ou com o mês seguinte -1
for f in df_cepea_consulta.index:
    if pd.isna(df_cepea_consulta['Data'][f]) == True:
        try:
            df_cepea_consulta.at[f, 'Data'] = df_cepea_consulta.at[f-1, 'Data'] + pd.DateOffset(months=1)
        except:
            df_cepea_consulta.at[f, 'Data'] = df_cepea_consulta.at[f+1, 'Data'] + pd.DateOffset(months=-1)

df_cepea_consulta

,Data,Valor
0,2021-11-01,"297,66"
1,2021-12-01,"320,90"
2,2022-01-01,"338,46"
3,2022-02-01,"340,29"
4,2022-03-01,"344,71"
5,2022-04-01,"335,06"
6,2022-05-01,"323,10"
7,2022-06-01,"317,96"
8,2022-07-01,"324,41"
9,2022-08-01,"313,39"


In [3]:
# Completa os campos de valores que não estejam preenchidos com o valor do mês anterior
df_cepea_consulta['Valor'].fillna(method='ffill', inplace=True)

# Converte a coluna 'Valor' para float
df_cepea_consulta['Valor'].replace(',', '.', regex = True, inplace = True)
df_cepea_consulta['Valor'] = df_cepea_consulta['Valor'].astype(float)

df_cepea_consulta

,Data,Valor
0,2021-11-01,297.66
1,2021-12-01,320.90
2,2022-01-01,338.46
3,2022-02-01,340.29
4,2022-03-01,344.71
5,2022-04-01,335.06
6,2022-05-01,323.10
7,2022-06-01,317.96
8,2022-07-01,324.41
9,2022-08-01,313.39


In [4]:
# Busca a série do IPCA com os dados necessários
ipca_mes = sgs.get({'ipca': 433}, start = df_cepea_consulta['Data'].iloc[0], end = df_cepea_consulta['Data'].iloc[-1])

# Reseta o índice e altera o nome das colunas para facilitar o merge
ipca_mes.reset_index(inplace=True)
ipca_mes.rename(columns = {'Date': 'Data', 'ipca': 'IPCA'}, inplace = True)

ipca_mes

,Data,IPCA
0,2021-11-01,0.95
1,2021-12-01,0.73
2,2022-01-01,0.54
3,2022-02-01,1.01
4,2022-03-01,1.62
5,2022-04-01,1.06
6,2022-05-01,0.47
7,2022-06-01,0.67
8,2022-07-01,-0.68
9,2022-08-01,-0.36


In [5]:
# Adiciona a coluna IPCA dando merge pela coluna 'Data'
df_cepea_consulta = pd.merge(df_cepea_consulta, ipca_mes, how = 'left', on = 'Data')

df_cepea_consulta

,Data,Valor,IPCA
0,2021-11-01,297.66,0.95
1,2021-12-01,320.90,0.73
2,2022-01-01,338.46,0.54
3,2022-02-01,340.29,1.01
4,2022-03-01,344.71,1.62
5,2022-04-01,335.06,1.06
6,2022-05-01,323.10,0.47
7,2022-06-01,317.96,0.67
8,2022-07-01,324.41,-0.68
9,2022-08-01,313.39,-0.36


In [6]:
# Calcula o campo IPCA acumulado com a soma acumulada do campo IPCA
df_cepea_consulta['IPCA_acumulado'] = df_cepea_consulta['IPCA'].cumsum()

df_cepea_consulta

,Data,Valor,IPCA,IPCA_acumulado
0,2021-11-01,297.66,0.95,0.95
1,2021-12-01,320.90,0.73,1.68
2,2022-01-01,338.46,0.54,2.22
3,2022-02-01,340.29,1.01,3.23
4,2022-03-01,344.71,1.62,4.85
5,2022-04-01,335.06,1.06,5.91
6,2022-05-01,323.10,0.47,6.38
7,2022-06-01,317.96,0.67,7.05
8,2022-07-01,324.41,-0.68,6.37
9,2022-08-01,313.39,-0.36,6.01


In [7]:
# Calcula o campo Real com o valor corrigido pelo IPCA de 12/2022
ipca_12_2022 = df_cepea_consulta.loc[df_cepea_consulta['Data'] == '2022-12-01', 'IPCA_acumulado'].iloc[0]
df_cepea_consulta['Real'] = df_cepea_consulta['Valor'] + (df_cepea_consulta['Valor'] * (ipca_12_2022 - df_cepea_consulta['IPCA_acumulado']) / 100)
df_cepea_consulta['Real'] = df_cepea_consulta['Real'].round(2)

df_cepea_consulta

,Data,Valor,IPCA,IPCA_acumulado,Real
0,2021-11-01,297.66,0.95,0.95,316.68
1,2021-12-01,320.90,0.73,1.68,339.06
2,2022-01-01,338.46,0.54,2.22,355.79
3,2022-02-01,340.29,1.01,3.23,354.28
4,2022-03-01,344.71,1.62,4.85,353.29
5,2022-04-01,335.06,1.06,5.91,339.85
6,2022-05-01,323.10,0.47,6.38,326.20
7,2022-06-01,317.96,0.67,7.05,318.88
8,2022-07-01,324.41,-0.68,6.37,327.56
9,2022-08-01,313.39,-0.36,6.01,317.56


In [8]:
# Carregamento do dataframe principal para a memória
df_boi_gordo = pd.read_csv('dags/data/boi_gordo_base.csv')

df_boi_gordo

,dt_cmdty,nome_cmdty,tipo_cmdty,cmdty_um,cmdty_vl_rs_um,cmdty_var_mes_perc,dt_etl
0,2021-11-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,263.95,2.664333,2021-12-14
1,2021-12-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,321.60,0.218413,2021-12-14
2,2022-02-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,338.46,0.052425,2022-02-02
3,2022-04-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,341.60,0.009277,2022-04-02
4,2022-06-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,314.60,-0.079040,2022-06-02
5,2022-07-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,485.90,0.544501,2022-07-02


In [9]:
# Converte as colunas necessárias
df_boi_gordo['dt_cmdty'] = pd.to_datetime(df_boi_gordo['dt_cmdty'], format='%Y-%m-%d')
df_boi_gordo['cmdty_vl_rs_um'] = df_boi_gordo['cmdty_vl_rs_um'].astype(float)
df_boi_gordo['cmdty_var_mes_perc'] = df_boi_gordo['cmdty_var_mes_perc'].astype(float)
df_boi_gordo['dt_etl'] = pd.to_datetime(df_boi_gordo['dt_etl'], format='%Y-%m-%d')

df_boi_gordo

,dt_cmdty,nome_cmdty,tipo_cmdty,cmdty_um,cmdty_vl_rs_um,cmdty_var_mes_perc,dt_etl
0,2021-11-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,263.95,2.664333,2021-12-14
1,2021-12-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,321.60,0.218413,2021-12-14
2,2022-02-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,338.46,0.052425,2022-02-02
3,2022-04-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,341.60,0.009277,2022-04-02
4,2022-06-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,314.60,-0.079040,2022-06-02
5,2022-07-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,485.90,0.544501,2022-07-02


In [10]:
# Modifica o 'df_cepea_consulta' para o merge criando um 'df_cepea_consulta_merge'
df_cepea_consulta_merge = df_cepea_consulta.rename(columns = {'Data': 'dt_cmdty'})

df_cepea_consulta_merge

,dt_cmdty,Valor,IPCA,IPCA_acumulado,Real
0,2021-11-01,297.66,0.95,0.95,316.68
1,2021-12-01,320.90,0.73,1.68,339.06
2,2022-01-01,338.46,0.54,2.22,355.79
3,2022-02-01,340.29,1.01,3.23,354.28
4,2022-03-01,344.71,1.62,4.85,353.29
5,2022-04-01,335.06,1.06,5.91,339.85
6,2022-05-01,323.10,0.47,6.38,326.20
7,2022-06-01,317.96,0.67,7.05,318.88
8,2022-07-01,324.41,-0.68,6.37,327.56
9,2022-08-01,313.39,-0.36,6.01,317.56


In [11]:
# Faz o merge pré-carregando o novo df_boi_gordo
df_boi_gordo = pd.merge(df_boi_gordo, df_cepea_consulta_merge[['dt_cmdty','Real']], how = 'left', on = 'dt_cmdty')

# Calculada a coluna 'cmdty_var_mes_perc', atualiza a coluna 'cmdty_vl_rs_um' e exclui a coluna temporária 'Real'
df_boi_gordo['cmdty_var_mes_perc'] = (df_boi_gordo['Real'] - df_boi_gordo['cmdty_vl_rs_um']) / df_boi_gordo['cmdty_vl_rs_um']
df_boi_gordo['cmdty_vl_rs_um'] = df_boi_gordo['Real']
del df_boi_gordo['Real']

df_boi_gordo

,dt_cmdty,nome_cmdty,tipo_cmdty,cmdty_um,cmdty_vl_rs_um,cmdty_var_mes_perc,dt_etl
0,2021-11-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,316.68,0.199773,2021-12-14
1,2021-12-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,339.06,0.054291,2021-12-14
2,2022-02-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,354.28,0.046741,2022-02-02
3,2022-04-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,339.85,-0.005123,2022-04-02
4,2022-06-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,318.88,0.013605,2022-06-02
5,2022-07-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,327.56,-0.325870,2022-07-02


In [12]:
# Prepara para salvar o dataframe
data_atual = datetime.now()
nome_arquivo_atualizado = 'boi_gordo_base_atualizado_' + data_atual.strftime('%d_%m_%Y') + '.parquet'

# Altera a coluna 'dt_etl' para a data atual
df_boi_gordo['dt_etl'] = pd.to_datetime(data_atual.strftime('%Y-%m-%d'), format='%Y-%m-%d')

# Salva o dataframe em parquet com as colunas necessárias
df_boi_gordo.to_parquet(nome_arquivo_atualizado)

In [13]:
# Verifica o arquivo salvo
df = pd.read_parquet(nome_arquivo_atualizado)

df

,dt_cmdty,nome_cmdty,tipo_cmdty,cmdty_um,cmdty_vl_rs_um,cmdty_var_mes_perc,dt_etl
0,2021-11-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,316.68,0.199773,2023-02-14
1,2021-12-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,339.06,0.054291,2023-02-14
2,2022-02-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,354.28,0.046741,2023-02-14
3,2022-04-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,339.85,-0.005123,2023-02-14
4,2022-06-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,318.88,0.013605,2023-02-14
5,2022-07-01,Boi_Gordo,Indicador do Boi Gordo CEPEA/B3,15 Kg/carcaça,327.56,-0.325870,2023-02-14
